In [1]:
import urllib
import flickrapi
import logging
from helper import get_urls
from helper import download_images
from time import time

In [2]:
import config
import flickrapi
import requests
import os
import pandas as pd
import pickle


def get_photos(image_tag):

    raw_photos = pd.DataFrame(columns=['description', 'latitude', 'longitude'])

    flickr = flickrapi.FlickrAPI(config.api_key, config.api_secret, format='parsed-json')
    # photos = flickr.walk(text=image_tag,        # it will search by image title and image tags
    #                      extras='url_c',     # get the urls for each size we want
    #                      privacy_filter=1,   # search only for public photos
    #                      per_page=50,
    #                      sort='relevance')   # we want what we are looking for to appear first
    #

    photos = flickr.photos.search(tags=image_tag,
                                  extras='description,geo',
                                  has_geo = 1
                                  )
    raw_photos = raw_photos.append(pd.DataFrame(photos['photos']['photo'])
                                   [['description', 'latitude', 'longitude']],
                                   ignore_index=True)
    with open('test.pkl', 'wb') as f:
        pickle.dump(raw_photos, f)

    return photos


def get_urls(image_tag, max):
    photos = get_photos(image_tag)
    counter = 0
    urls = []

    for photo in photos:
        if counter < max:
            url = photo.get('url_c')
            if url:
                urls.append(url)
                counter += 1
            # if no url for the desired sizes then try with the next photo
        else:
            break
    return urls


def create_folder(path):
    if not os.path.isdir(path):
        os.makedirs(path)


def download_images(urls, path):
    create_folder(path)
    total = 0

    for url in urls:
        image_name = str(total).zfill(6) + '.jpg'
        image_path = os.path.join(path, image_name)

        if not os.path.isfile(image_path):  # ignore if already downloaded
            response = requests.get(url, stream=True)

            with open(image_path, 'wb') as outfile:
                outfile.write(response.content)

        total += 1

    logging.info('download completed.')

In [3]:
keyword = 'camping'
get_photos(keyword)
with open('test.pkl', 'rb') as f:
    test = pickle.load(f)
    f.close()

In [4]:
# print(test)

In [5]:
keyword = 'beach'
max_num = 20
urls = get_urls(keyword, max_num)

In [6]:
start = time()
download_images(urls, "data/%s" %keyword)
end = time()

print('Completed in %.2e seconds.' %(end-start))

Completed in 3.45e+00 seconds.


In [7]:
# from sklearn.utils import Bunch